In [1]:
# Load libraries
import numpy as np
import pandas as pd

In [3]:
# Load dataset
TwtDs = pd.read_csv('C:\\Users\\Meng\\OneDrive\\Documents\\tamu\\DataScience\\twitter-user-gender-classification\\gender-classifier-DFE-791531.csv',  na_values=['.'], encoding = "ISO-8859-1")

In [4]:
# shape
print(TwtDs.shape) 

(20050, 26)


In [5]:
# head
print(TwtDs.head(10))

    _unit_id _golden _unit_state  _trusted_judgments _last_judgment_at  \
0  815719226   False   finalized                   3    10/26/15 23:24   
1  815719227   False   finalized                   3    10/26/15 23:30   
2  815719228   False   finalized                   3    10/26/15 23:33   
3  815719229   False   finalized                   3    10/26/15 23:10   
4  815719230   False   finalized                   3     10/27/15 1:15   
5  815719231   False   finalized                   3     10/27/15 1:47   
6  815719232   False   finalized                   3     10/27/15 1:57   
7  815719233   False   finalized                   3    10/26/15 23:48   
8  815719234   False   finalized                   3     10/27/15 1:52   
9  815719235   False   finalized                   3     10/27/15 1:49   

   gender  gender:confidence profile_yn  profile_yn:confidence  \
0    male             1.0000        yes                    1.0   
1    male             1.0000        yes              

In [6]:
# check current data type
TwtDs.dtypes

_unit_id                   int64
_golden                     bool
_unit_state               object
_trusted_judgments         int64
_last_judgment_at         object
gender                    object
gender:confidence        float64
profile_yn                object
profile_yn:confidence    float64
created                   object
description               object
fav_number                 int64
gender_gold               object
link_color                object
name                      object
profile_yn_gold           object
profileimage              object
retweet_count              int64
sidebar_color             object
text                      object
tweet_coord               object
tweet_count                int64
tweet_created             object
tweet_id                 float64
tweet_location            object
user_timezone             object
dtype: object

In [7]:
# Dataset Info
TwtDs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
_unit_id                 20050 non-null int64
_golden                  20050 non-null bool
_unit_state              20050 non-null object
_trusted_judgments       20050 non-null int64
_last_judgment_at        20000 non-null object
gender                   19953 non-null object
gender:confidence        20024 non-null float64
profile_yn               20050 non-null object
profile_yn:confidence    20050 non-null float64
created                  20050 non-null object
description              16306 non-null object
fav_number               20050 non-null int64
gender_gold              50 non-null object
link_color               20050 non-null object
name                     20050 non-null object
profile_yn_gold          50 non-null object
profileimage             20050 non-null object
retweet_count            20050 non-null int64
sidebar_color            20050 non-null object
text    

In [10]:
# gender distribution
print(TwtDs.groupby('gender').size())
#print(TwtDs.groupby('gender').sum())

gender
brand      5942
female     6700
male       6194
unknown    1117
dtype: int64


In [11]:
TwtHd = TwtDs.head(100)

In [12]:
TwtHd.shape

(100, 26)

In [14]:
TwtHd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 26 columns):
_unit_id                 100 non-null int64
_golden                  100 non-null bool
_unit_state              100 non-null object
_trusted_judgments       100 non-null int64
_last_judgment_at        100 non-null object
gender                   100 non-null object
gender:confidence        100 non-null float64
profile_yn               100 non-null object
profile_yn:confidence    100 non-null float64
created                  100 non-null object
description              93 non-null object
fav_number               100 non-null int64
gender_gold              0 non-null object
link_color               100 non-null object
name                     100 non-null object
profile_yn_gold          0 non-null object
profileimage             100 non-null object
retweet_count            100 non-null int64
sidebar_color            100 non-null object
text                     100 non-null object
tweet

In [16]:
TwtHd['txtwd']=TwtHd.text.str.split()

C:\Users\Meng\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
TwtHd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 27 columns):
_unit_id                 100 non-null int64
_golden                  100 non-null bool
_unit_state              100 non-null object
_trusted_judgments       100 non-null int64
_last_judgment_at        100 non-null object
gender                   100 non-null object
gender:confidence        100 non-null float64
profile_yn               100 non-null object
profile_yn:confidence    100 non-null float64
created                  100 non-null object
description              93 non-null object
fav_number               100 non-null int64
gender_gold              0 non-null object
link_color               100 non-null object
name                     100 non-null object
profile_yn_gold          0 non-null object
profileimage             100 non-null object
retweet_count            100 non-null int64
sidebar_color            100 non-null object
text                     100 non-null object
tweet

In [21]:
TwtHd.txtwd

0     [Robbie, E, Responds, To, Critics, After, Win,...
1     [ÛÏIt, felt, like, they, were, my, friends, a...
2     [i, absolutely, adore, when, louis, starts, th...
3     [Hi, @JordanSpieth, -, Looking, at, the, url, ...
4     [Watching, Neighbours, on, Sky+, catching, up,...
5     [Ive, seen, people, on, the, train, with, lamp...
6     [@BpackEngineer, Thank, you, for, your, patien...
7     [Gala, Bingo, clubs, bought, for, å£241m:, The...
8     [@_Aphmau_, the, pic, defines, all, mcd, fangi...
9     [@Evielady, just, how, lovely, is, the, tree, ...
10    [how, are, you, taking, care, of, yourself?, h...
11    [MTG, Deals, 1x, Rank-Up-Magic, -, The, Sevent...
12    [Just, put, my, ass, on, the, line, for, you, ...
13    [https://t.co/z4sbWUugd8, What, the, Nation, W...
14    [will, i, even, need, sound, effects, for, the...
15    [https://t.co/EROGWTFTYo, It's, a, glow, of, s...
16    [@giannaaa28, lmao, _Ù÷â_Ù÷â, dude, I'm, hella...
17    [@coolyazzy94, Ditto, -, I'm, still, learn

In [22]:
TwtHd.txtwd[5]

['Ive',
 'seen',
 'people',
 'on',
 'the',
 'train',
 'with',
 'lamps,',
 'chairs,',
 'tvs',
 'etc',
 'https://t.co/w6zf4pVM4I']

In [23]:
# gender distribution in the sample
print(TwtHd.groupby('gender').size())

gender
brand      28
female     33
male       37
unknown     2
dtype: int64
